## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np
import requests
import gmaps

# Import API key
import sys  
sys.path.insert(0, '../')

import config
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Rapid Valley,US,44.0625,-103.1463,82.63,51,100,4.00,overcast clouds
1,1,Makhachkala,RU,42.9764,47.5024,80.58,68,0,15.73,clear sky
2,2,Rikitea,PF,-23.1203,-134.9692,72.81,78,13,17.40,light rain
3,3,Imbituba,BR,-28.2400,-48.6703,71.85,83,100,18.99,overcast clouds
4,4,Zelenets,RU,61.8849,50.7489,57.45,59,18,2.71,few clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 65
What is the maximum temperature you would like for your trip? 80


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,Rikitea,PF,-23.1203,-134.9692,72.81,78,13,17.40,light rain
3,3,Imbituba,BR,-28.2400,-48.6703,71.85,83,100,18.99,overcast clouds
6,6,Hualmay,PE,-11.0964,-77.6139,66.18,71,32,10.31,scattered clouds
12,12,Nyimba,ZM,-14.5566,30.8149,74.77,26,8,2.80,clear sky
13,13,Kapaa,US,22.0752,-159.3190,75.18,87,27,3.00,scattered clouds
14,14,Yellowknife,CA,62.4560,-114.3525,68.56,56,0,12.66,clear sky
15,15,Hayden,US,47.7660,-116.7866,74.79,48,0,9.22,clear sky
16,16,Ponta Do Sol,PT,32.6667,-17.1000,79.72,72,4,2.17,clear sky
18,18,Goderich,CA,43.7501,-81.7165,76.91,43,6,6.91,clear sky
20,20,Fereydunshahr,IR,32.9410,50.1210,69.53,14,0,5.06,clear sky


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                288
City                   288
Country                288
Lat                    288
Lng                    288
Max Temp               288
Humidity               288
Cloudiness             288
Wind Speed             288
Current Description    288
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.head(5)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,Rikitea,PF,-23.1203,-134.9692,72.81,78,13,17.40,light rain
3,3,Imbituba,BR,-28.2400,-48.6703,71.85,83,100,18.99,overcast clouds
6,6,Hualmay,PE,-11.0964,-77.6139,66.18,71,32,10.31,scattered clouds
12,12,Nyimba,ZM,-14.5566,30.8149,74.77,26,8,2.80,clear sky
13,13,Kapaa,US,22.0752,-159.3190,75.18,87,27,3.00,scattered clouds


In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Rikitea,PF,72.81,light rain,-23.1203,-134.9692,
3,Imbituba,BR,71.85,overcast clouds,-28.2400,-48.6703,
6,Hualmay,PE,66.18,scattered clouds,-11.0964,-77.6139,
12,Nyimba,ZM,74.77,clear sky,-14.5566,30.8149,
13,Kapaa,US,75.18,scattered clouds,22.0752,-159.3190,
14,Yellowknife,CA,68.56,clear sky,62.4560,-114.3525,
15,Hayden,US,74.79,clear sky,47.7660,-116.7866,
16,Ponta Do Sol,PT,79.72,clear sky,32.6667,-17.1000,
18,Goderich,CA,76.91,clear sky,43.7501,-81.7165,
20,Fereydunshahr,IR,69.53,clear sky,32.9410,50.1210,


In [8]:
hotel_df.count()

City                   288
Country                288
Max Temp               288
Current Description    288
Lat                    288
Lng                    288
Hotel Name             288
dtype: int64

In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [10]:
# 7. Drop the rows where there is no Hotel Name.
nan_hotel_df = hotel_df.replace(r'^s*$', float('NaN'), regex = True)

In [11]:
clean_hotel_df = nan_hotel_df.dropna()

In [12]:
clean_hotel_df.count()

City                   271
Country                271
Max Temp               271
Current Description    271
Lat                    271
Lng                    271
Hotel Name             271
dtype: int64

In [13]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [14]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description} {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]


In [15]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))